In [1]:
import matplotlib as mpl #画图用的库
import matplotlib.pyplot as plt
#下面这一句是为了可以在notebook中画图
%matplotlib inline
import numpy as np
import sklearn   #机器学习算法库
import pandas as pd #处理数据的库   
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras   #使用tensorflow中的keras
#import keras #单纯的使用keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, sklearn, pd, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.0
sklearn 0.21.3
pandas 0.25.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
#引用位于sklearn数据集中的房价预测数据集
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
#print(housing.DESCR) #数据集的描述
print(housing.data.shape) #相当于 x
print(housing.target.shape) #相当于 y

(20640, 8)
(20640,)


In [3]:
#用sklearn中专门用于划分训练集和测试集的方法
from sklearn.model_selection import train_test_split

#train_test_split默认将数据划分为3:1，我们可以通过修改test_size值来改变数据划分比例(默认0.25，即3:1)
#将总数乘以test_size就表示test测试集、valid验证集数量
#将数据集整体拆分为train_all和test数据集
x_train_all,x_test, y_train_all,y_test = train_test_split(housing.data, housing.target, random_state=7)
#将train_all数据集拆分为train训练集和valid验证集
x_train,x_valid, y_train,y_valid = train_test_split(x_train_all, y_train_all, random_state=11)

print(x_train_all.shape,y_train_all.shape)
print(x_test.shape, y_test.shape)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

(15480, 8) (15480,)
(5160, 8) (5160,)
(11610, 8) (11610,)
(3870, 8) (3870,)


In [4]:
#训练数据归一化处理
# x = (x - u)/std  u为均值，std为方差
from sklearn.preprocessing import StandardScaler #使用sklearn中的StandardScaler实现训练数据归一化

scaler = StandardScaler()#初始化一个scaler对象
x_train_scaler = scaler.fit_transform(x_train)#x_train已经是二维数据了，无需astype转换
x_valid_scaler = scaler.transform(x_valid)
x_test_scaler  = scaler.transform(x_test)

In [5]:
print(housing.feature_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [6]:
import shutil

output_dir = "generate_csv"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir) #存在该文件则先删除该文件夹，保证每次运行创建的文件夹都是新的
os.mkdir(output_dir)

def save_to_csv(output_dir, data, name_prefix, header=None, n_parts=10):
    path_format = os.path.join(output_dir, "{}_{:02d}.csv")#在大括号里面填充数据生成文件名
    filenames = []
    
    #enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中
    # file_idx表示生成的n_partes个文件的索引，row_indices表示每一个文件内的行索引
    for file_idx, row_indices in enumerate(np.array_split(np.arange(len(data)), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)#path_format传入两个参数组成文件名
        filenames.append(part_csv) #append生成最终的文件
        with open(part_csv, "wt", encoding="utf-8") as f: #打开这个文件并填充内容
            if header is not None:
                f.write(header + "\n")
            for row_index in row_indices: #这里遍历行索引,对每一个文件进行填充
                f.write(",".join([repr(col) for col in data[row_index]]))#对每一行中的每一列进行填充
                f.write('\n')
    return filenames

#因为save_to_csv只有一个data参数，所以我们将(x, y)按行进行合并
train_data = np.c_[x_train_scaler, y_train]#np.c_函数可以将数据按行进行合并merge
valid_data = np.c_[x_valid_scaler, y_valid]
test_data  = np.c_[x_test_scaler , y_test ]

#生成表头header，即每一列的数据前面的名称    
header_cols = housing.feature_names + ["HouseValue"]
print(header_cols)
header_str = ",".join(header_cols)

#训练集数据总数为 11610，n_parts表示平均分为n份
train_filenames = save_to_csv(output_dir, train_data, "train", header_str, n_parts=20)

valid_filenames = save_to_csv(output_dir, valid_data, "valid", header_str, n_parts=10)

test_filenames  = save_to_csv(output_dir, test_data, "test", header_str, n_parts=10)


['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'HouseValue']


In [7]:
import pprint

print("train_files:")
pprint.pprint(train_filenames)

print("valid_files:")
pprint.pprint(valid_filenames)

print("test_files:")
pprint.pprint(test_filenames)


train_files:
['generate_csv/train_00.csv',
 'generate_csv/train_01.csv',
 'generate_csv/train_02.csv',
 'generate_csv/train_03.csv',
 'generate_csv/train_04.csv',
 'generate_csv/train_05.csv',
 'generate_csv/train_06.csv',
 'generate_csv/train_07.csv',
 'generate_csv/train_08.csv',
 'generate_csv/train_09.csv',
 'generate_csv/train_10.csv',
 'generate_csv/train_11.csv',
 'generate_csv/train_12.csv',
 'generate_csv/train_13.csv',
 'generate_csv/train_14.csv',
 'generate_csv/train_15.csv',
 'generate_csv/train_16.csv',
 'generate_csv/train_17.csv',
 'generate_csv/train_18.csv',
 'generate_csv/train_19.csv']
valid_files:
['generate_csv/valid_00.csv',
 'generate_csv/valid_01.csv',
 'generate_csv/valid_02.csv',
 'generate_csv/valid_03.csv',
 'generate_csv/valid_04.csv',
 'generate_csv/valid_05.csv',
 'generate_csv/valid_06.csv',
 'generate_csv/valid_07.csv',
 'generate_csv/valid_08.csv',
 'generate_csv/valid_09.csv']
test_files:
['generate_csv/test_00.csv',
 'generate_csv/test_01.csv',
 'ge

In [8]:
###读取 csv 文件并形成一个datasets
#1.filename -> dataset
#2.read file -> dataset ->datasets ->merge
#3.parse csv

#首先生成一个文件名的数据集
filename_dataset = tf.data.Dataset.list_files(train_filenames)

for item in filename_dataset:
    print(item)

    
#读取每一个文件形成最终的数据集
n_readers = 5
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1),#skip(1)表示跳过第一行
    cycle_length = n_readers#每个文件读取前面5行数据(skip(1)表示已经除去了header)
)

for line in dataset.take(15):#take(15)表示只读取前面15个
    print(line.numpy())

tf.Tensor(b'generate_csv/train_18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_04.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_19.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', 

In [9]:
#解析csv中的字符串str，record_defaults表示 字符串中每个元素对应的数据类型
# tf.io.devode_csv(str, record_defaults)

sample_str      = '1,2,3,4,5'#解析samole_str这个字符串
record_defaults = [tf.constant(0, dtype=tf.int32)] * 5
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)

sample_str2      = '1,2,3,4,5'#解析samole_str这个字符串
record_defaults2 = [tf.constant(0, dtype=tf.int32),
                    0,
                    np.nan,
                    "hello",
                    tf.constant([])]
parsed_fields2 = tf.io.decode_csv(sample_str2, record_defaults2)
print(parsed_fields2)

#传入 空的字符串
try:
    parsed_fields2 = tf.io.decode_csv(',,,,', record_defaults2)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

#传入 字符串files数量大于record_defaults    
try:
    parsed_fields2 = tf.io.decode_csv('1,2,3,4,5,6,7', record_defaults2)
except tf.errors.InvalidArgumentError as ex:
    print(ex)    

[<tf.Tensor: id=96, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=97, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=98, shape=(), dtype=int32, numpy=3>, <tf.Tensor: id=99, shape=(), dtype=int32, numpy=4>, <tf.Tensor: id=100, shape=(), dtype=int32, numpy=5>]
[<tf.Tensor: id=107, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=108, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=109, shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: id=110, shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: id=111, shape=(), dtype=float32, numpy=5.0>]
Field 4 is required but missing in record 0! [Op:DecodeCSV]
Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [10]:
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields
    parse_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parse_fields[:-1])#前8个数据为x，需要区分出来
    y = tf.stack(parse_fields[-1:])#倒数第一个数据
    return x, y
parse_csv_line('-0.9974222662636643,1.2333642636907922,-0.7577192870888144,-0.011109251557751528,-0.23003784053222506,0.05487422342718872,-0.757726890467217,0.7065494722340417,1.739',n_fields=9)

(<tf.Tensor: id=131, shape=(8,), dtype=float32, numpy=
 array([-0.9974223 ,  1.2333642 , -0.7577193 , -0.01110925, -0.23003784,
         0.05487422, -0.7577269 ,  0.70654947], dtype=float32)>,
 <tf.Tensor: id=132, shape=(1,), dtype=float32, numpy=array([1.739], dtype=float32)>)

In [11]:
###读取 csv 文件并形成一个datasets
#1.filename -> dataset
#2.read file -> dataset ->datasets ->merge
#3.parse csv

def csv_reader_dataset(filenames, n_readers=5, 
                       batch_size=32, n_parse_threads=5, 
                       shuffle_buffer_size=10000):
    
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()#repeat()表示 数据集遍历的次数，无参数表示遍历无数次
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)#将数据打乱，数值越大，混乱程度越大
    dataset = dataset.map(parse_csv_line, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)#一次性取出 batch_size 个数据
    return dataset
    
train_set = csv_reader_dataset(train_filenames, batch_size=3)
for x_batch, y_batch in train_set.take(2):
    print("x:")
    pprint.pprint(x_batch)
    print("y:")
    pprint.pprint(y_batch)

x:
<tf.Tensor: id=216, shape=(3, 8), dtype=float32, numpy=
array([[ 0.15782312,  0.4323619 ,  0.3379948 , -0.01588031, -0.37338907,
        -0.05305246,  0.80061346, -1.2359096 ],
       [-0.66722274, -0.04823952,  0.34529406,  0.53826684,  1.8521839 ,
        -0.06112538, -0.8417093 ,  1.5204847 ],
       [ 2.5150437 ,  1.0731637 ,  0.5574401 , -0.17273512, -0.6129126 ,
        -0.01909157, -0.5710993 , -0.02749031]], dtype=float32)>
y:
<tf.Tensor: id=217, shape=(3, 1), dtype=float32, numpy=
array([[3.169  ],
       [1.59   ],
       [5.00001]], dtype=float32)>
x:
<tf.Tensor: id=218, shape=(3, 8), dtype=float32, numpy=
array([[-1.0775077 , -0.4487407 , -0.5680568 , -0.14269263, -0.09666677,
         0.12326469, -0.31448638, -0.4818959 ],
       [ 0.48530516, -0.8492419 , -0.06530126, -0.02337966,  1.4974351 ,
        -0.07790658, -0.90236324,  0.78145146],
       [ 2.2878418 , -1.890545  ,  0.66071063, -0.14964779, -0.06672633,
         0.44788057, -0.5337738 ,  0.56673235]], dtype=fl

In [12]:
batch_size = 32
train_set = csv_reader_dataset(train_filenames, batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames, batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,   batch_size = batch_size)

In [13]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu",input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="adam")


callbacks = [
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)
]

history=model.fit(train_set,
                  validation_data = valid_set,
                  steps_per_epoch = 11160 // batch_size,
                  validation_steps = 3870 // batch_size,
                  epochs = 100,
                  callbacks=callbacks)


Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 2s 5ms/step - loss: 1.4231 - val_loss: 0.7870
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 0.6068 - val_loss: 0.5677
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4762 - val_loss: 0.4680
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4172 - val_loss: 0.4236
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3986 - val_loss: 0.4127
Epoch 6/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3904 - val_loss: 0.3968
Epoch 7/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3813 - val_loss: 0.3894
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3831 - val_loss: 0.3837
Epoch 9/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3738 - val_loss: 0.3859
Epoch 10/100
348/348 [=========================

In [14]:
model.evaluate(test_set, steps=5160 // batch_size)

161/161 [==============================] - 0s 1ms/step - loss: 0.3512


0.35118211963161916